<a href="https://colab.research.google.com/github/Mhobo/SimNets/blob/main/DML_SSL_Small_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchinfo
!pip install torch
!pip install sklearn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchinfo import summary
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
from torch.functional import norm
import random as rnd
import argparse
import os
import time
import math
import json
import torch
from torch.autograd import Variable as V
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
from PIL import Image
import torch.nn as nn

In [ ]:
# Image preprocessing modules - FashionMNIST (28x28 greyscale)
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(28),
    transforms.ToTensor()])

# FashionMNIST dataset
# Dataset objects and Dataloader objects are objects Python defines
train_dataset = torchvision.datasets.FashionMNIST(root='data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),  #replace with = transform after
                                             download=True)

test_dataset = torchvision.datasets.FashionMNIST(root='data/',
                                            train=False,#need comma here to get transforms back
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=128, 
                                            shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=128, shuffle=False)
                                           

In [ ]:
# already, shuffled, choose every 10th entry
def Corrupter(train_dataset,corruption_ratio):
  Changed = range(0,6000,corruption_ratio)
  Changed_dataset = train_dataset
  # set new labels that are wrong
  New_Values = []
  for i in Changed_dataset.targets[Changed]:
    Label_Options = list(range(0,10))
    Label_Options.remove(Changed_dataset.targets[Changed[i]])
    New_Value = rnd.choice(Label_Options)
    Changed_dataset.targets[Changed[i]] = New_Value
    New_Values.append(New_Value)

    Changed_loader = torch.utils.data.DataLoader(dataset=Changed_dataset,
                                            batch_size=128, 
                                            shuffle=True)
    return(Changed_loader)

In [ ]:
# the layers we want in between, need somewhere to check dimension of dataset?
class MLP(nn.Module):
  def __init__(self,layer_widths): 
 
    super().__init__()
    self.m = len(layer_widths) #self stores it as characteristic of this NN
    self.layers = [] # Empty list to store layers
    self.non_lin = nn.ReLU() # nn.ReLU isn't the function, but the function that constructs a relu (by saying =)
    for i in range(self.m-1): #does this make m-1 iterations?
      self.layers.append(nn.Linear(layer_widths[i], layer_widths[i+1])) # Need matrix maths to work, so have (a,b) thenn (b,c)
      self.add_module("layer"+str(i), self.layers[-1])
 
  def forward(self, x):
    z = x
    for i in range(self.m - 1):
      y = z
      z = self.non_lin(self.layers[i](z))
    return z,y
# Probably better off hardcoding, sanity check and because the layers are designed with the dataset in mind

In [ ]:
m = [(28*28),50,50,50,10]
net = MLP(m) # creating the network
net.rot_pred = nn.Linear(50, 4)

net2 = MLP(m) # creating the network
net2.rot_pred = nn.Linear(50, 4)

device = torch.device('cuda')
num_epochs = 50
learning_rate = 0.0004
reg_constant = 0.000005
L_norm = 1

net.cuda()
net2.cuda()



MLP(
  (non_lin): ReLU()
  (layer0): Linear(in_features=784, out_features=50, bias=True)
  (layer1): Linear(in_features=50, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=50, bias=True)
  (layer3): Linear(in_features=50, out_features=10, bias=True)
  (rot_pred): Linear(in_features=50, out_features=4, bias=True)
)

In [ ]:
optimizer = torch.optim.Adam(list(net.parameters()) + list(net2.parameters()), lr= learning_rate) # An optimiser object
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

total_step = len(train_loader)
curr_lr = learning_rate


In [ ]:
def train(corruption_ratio = 10, T=1, scheduler = scheduler,no_ss = True):
      net.train()
      net2.train()
      loss_avg = 0.0
      loss_avg2 = 0.0
      torch.set_grad_enabled(True)

      for i, (bx, by) in enumerate(train_loader): #Corrupter(train_dataset,corruption_ratio)
          bx, by = torch.flatten(bx.to(device),start_dim = 1), by.cuda()
          #

        
        #for model in models: (indent stuff below)
        # forward
          logits,_ = net(bx * 2 -1) # change 'net' to 'models'? could also leave it as kind of scrappy code and manually write 'net' and 'nets'
          logits2,_ = net2(bx *2 -1)
              
          loss = F.cross_entropy(logits, by)
          loss2 = F.cross_entropy(logits2, by)
            # backward

              
          if not no_ss:
              curr_batch_size = bx.size(0)
              by_prime = torch.cat((torch.zeros(bx.size(0)), torch.ones(bx.size(0)),
                                  2*torch.ones(bx.size(0)), 3*torch.ones(bx.size(0))), 0).long()

              bx = torch.cat((bx, torch.rot90(bx,1, dims=[0,1]),torch.rot90(bx, 2, dims=[0, 1]), torch.rot90(bx, 3, dims=[0, 1])), 0)
              bx, by_prime = bx.cuda(), by_prime.cuda()

              _, pen = net(bx * 2 - 1)
              _, pen2 = net2(bx * 2 - 1)

                #pen = pen.cuda()
                #pen2 = pen2.cuda()


              loss += 0.5 * F.cross_entropy(net.rot_pred(pen), by_prime)
              loss2 += 0.5 * F.cross_entropy(net2.rot_pred(pen2), by_prime)


            # KL loss, set to 0 for now
            # KL loss, set to 0 for now, this part just gets ignored? /0 gives no error but also doesn't train
            # Should ask it to state KL loss over time, compare to other parts
            
              DML_loss = T*F.cross_entropy(net.rot_pred(pen), F.softmax(net2.rot_pred(pen2).detach(),dim=1))
              loss += DML_loss 
            
                #kl_loss2 += 0.01*loss_kl(F.log_softmax(net2.rot_pred(pen2),dim = 1), F.softmax(net.rot_pred(pen),dim = 1))
              DML_loss2 = T*F.cross_entropy(net2.rot_pred(pen2), F.softmax(net.rot_pred(pen).detach(),dim=1))
              loss2 += DML_loss2 

          optimizer.zero_grad()
          loss3 = loss + loss2
          loss3.backward()  
          optimizer.step()       
        
          loss_avg = loss_avg * 0.95 + loss.item() * 0.05
          loss_avg2 = loss_avg2 * 0.95 + loss2.item() * 0.05

          train_loss = loss_avg
          train_loss2 = loss_avg2
      
      scheduler.step(loss3)
        # exponential moving average
          #Avg_DML_Loss = 0.5*(DML_loss.item() + DML_loss2.item())
          #print(train_loss,train_loss2)#,Avg_DML_Loss

In [ ]:
# test function (forward only)
def test():
    torch.set_grad_enabled(False)
    net.eval()
    net2.eval()
    loss_avg = 0.0
    correct = 0

    loss_avg2 = 0.0
    correct2 = 0

    with torch.autocast(device_type = 'cuda', dtype = torch.float16):
        for bx, by in test_loader:
            bx, by = torch.flatten(bx.to(device),start_dim = 1), by.cuda()

        # forward
            logits, pen = net(bx *2 -1) # Does the -1 mean look at the penultimate layer? Or is it just the ','
            loss = F.cross_entropy(logits, by)

            logits2, pen2 = net2(bx * 2 -1)
            loss2 = F.cross_entropy(logits2, by)

        
        # accuracy
            pred = logits.data.max(1)[1]
            correct += pred.eq(by.data).sum().item()

            pred2 = logits2.data.max(1)[1]
            correct2 += pred2.eq(by.data).sum().item()


        # test loss average
            loss_avg += loss.item()
            loss_avg2 += loss2.item()

        test_loss = loss_avg / len(test_loader)
        test_accuracy = correct / len(test_loader.dataset)
        test_loss2 = loss_avg2 / len(test_loader)
        test_accuracy2 = correct2 / len(test_loader.dataset)
        print(test_accuracy,test_accuracy2)
    torch.set_grad_enabled(True)

In [ ]:
for epoch in range(num_epochs):

    begin_epoch = time.time()
    train(scheduler=scheduler)
    test()
    print('Epoch', epoch, '| Time Spent:', round(time.time() - begin_epoch, 2))
    

0.6041 0.5323
Epoch 0 | Time Spent: 11.73
0.6041 0.5323
Epoch 1 | Time Spent: 11.64
0.6041 0.5323
Epoch 2 | Time Spent: 11.5


KeyboardInterrupt: ignored